In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import os
from tensorflow.keras import regularizers
from tensorflow.keras import datasets, regularizers

(train_images, train_labels), (test_images, test_labels)  = datasets.mnist.load_data()

train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))


train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Use part of the training data for validation
)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow(train_images, train_labels, batch_size=32, subset='training')
val_generator = train_datagen.flow(train_images, train_labels, batch_size=32, subset='validation')
test_generator = test_datagen.flow(test_images, test_labels, batch_size=32)

11490434/11490434 [==============================] - 0s 0us/step


In [2]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')  # 10 classes for MNIST
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 batch_normalization (Batch  (None, 28, 28, 64)        256       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 batch_normalization_1 (Bat  (None, 28, 28, 64)        256       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 64)        0         
 D)                                                              
                                                        

In [4]:
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=20)

Epoch 1/20
1500/1500 [==============================] - 37s 18ms/step - loss: 1.1038 - accuracy: 0.8654 - val_loss: 0.6538 - val_accuracy: 0.9209
Epoch 2/20
1500/1500 [==============================] - 28s 19ms/step - loss: 0.4916 - accuracy: 0.9467 - val_loss: 0.7671 - val_accuracy: 0.8322
Epoch 3/20
1500/1500 [==============================] - 29s 19ms/step - loss: 0.4134 - accuracy: 0.9554 - val_loss: 0.3658 - val_accuracy: 0.9687
Epoch 4/20
1500/1500 [==============================] - 27s 18ms/step - loss: 0.3927 - accuracy: 0.9597 - val_loss: 0.3626 - val_accuracy: 0.9728
Epoch 5/20
1500/1500 [==============================] - 27s 18ms/step - loss: 0.3867 - accuracy: 0.9617 - val_loss: 0.3307 - val_accuracy: 0.9762
Epoch 6/20
1500/1500 [==============================] - 26s 18ms/step - loss: 0.3503 - accuracy: 0.9671 - val_loss: 0.3199 - val_accuracy: 0.9760
Epoch 7/20
1500/1500 [==============================] - 28s 19ms/step - loss: 0.3445 - accuracy: 0.9685 - val_loss: 0.3092 -

In [5]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

313/313 [==============================] - 2s 5ms/step - loss: 0.2553 - accuracy: 0.9696
Test accuracy: 0.9696000218391418


In [6]:
model.save('mnistmodel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
